In [1]:
from uit.testing_utils import *
mock_client = MockClient()

mock_script = PbsScript(
    name='mock_script',
    project_id='mock_project_id',
    num_nodes=1,
    processes_per_node=44,
    max_time='00:00:01'
)

mock_job = MockPbsJob(mock_script, client=mock_client, label='mock')

mock_array_script = PbsScript(
    name='mock_script',
    project_id='mock_project_id',
    num_nodes=1,
    processes_per_node=44,
    max_time='00:00:01',
    array_indices=(0, 2)
)

mock_array_job = MockPbsArrayJob(
    script=mock_array_script,
    client=mock_client,
    label='mock'
)
mock_array_job._job_id = '[0]'
mock_array_job._remote_workspace_id = 'mock_workspace'

initializing array job


In [2]:
print(mock_array_job._job_id)

[0]


In [3]:
print(mock_array_job.sub_jobs)

[<PbsArraySubJob name=mock_script id=[0]>, <PbsArraySubJob name=mock_script id=[0]>, <PbsArraySubJob name=mock_script id=[0]>]


In [4]:
i = 0
for job in mock_array_job.sub_jobs:
    job._job_id = job._job_id + f'[{i+1}]'
    i += 1
for job in mock_array_job.sub_jobs:
    print ('job id is',job._job_id)

job id is [0][1]
job id is [0][2]
job id is [0][3]


In [5]:
statuses = [None,None,None,None]
dict = {'status': 'R','job_id':'[0]','elap_time':'00:05'}
for i in range(len(statuses)):
    statuses[i] = dict.copy()
for i in range(len(mock_array_job.sub_jobs)):
    statuses[i+1]['job_id'] = mock_array_job.sub_jobs[i]._job_id
print (statuses) 

[{'status': 'R', 'job_id': '[0]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][1]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][2]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:05'}]


In [6]:
for job in mock_array_job.sub_jobs: print(job.qstat)

None
None
None


In [7]:
print('sending',statuses)
mock_array_job.update_status(statuses)
for job in mock_array_job.sub_jobs: print(job.qstat)

sending [{'status': 'R', 'job_id': '[0]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][1]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][2]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:05'}]
remaining jobs are [<MockPbsArrayJob name=mock_script id=[0]>, <PbsArraySubJob name=mock_script id=[0][1]>, <PbsArraySubJob name=mock_script id=[0][2]>, <PbsArraySubJob name=mock_script id=[0][3]>]
{'status': 'R', 'job_id': '[0][1]', 'elap_time': '00:05'}
{'status': 'R', 'job_id': '[0][2]', 'elap_time': '00:05'}
{'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:05'}


In [8]:
statuses[1]['status']='F'
print(statuses)

[{'status': 'R', 'job_id': '[0]', 'elap_time': '00:05'}, {'status': 'F', 'job_id': '[0][1]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][2]', 'elap_time': '00:05'}, {'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:05'}]


In [9]:
mock_array_job.update_status(statuses)
for job in mock_array_job.sub_jobs: print(job.qstat)

remaining jobs are [<MockPbsArrayJob name=mock_script id=[0]>, <PbsArraySubJob name=mock_script id=[0][1]>, <PbsArraySubJob name=mock_script id=[0][2]>, <PbsArraySubJob name=mock_script id=[0][3]>]
{'status': 'F', 'job_id': '[0][1]', 'elap_time': '00:05'}
{'status': 'R', 'job_id': '[0][2]', 'elap_time': '00:05'}
{'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:05'}


In [10]:
print(mock_array_job.sub_jobs[0].status)

F


In [11]:
statuses[2]={'job_id':'[0][2]','status':''}
del statuses[
print(statuses)

[{'status': 'R', 'job_id': '[0]', 'elap_time': '00:05'}, {'status': 'F', 'job_id': '[0][1]', 'elap_time': '00:05'}, {'job_id': '[0][2]', 'status': ''}, {'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:05'}]


In [12]:
mock_array_job.update_status(statuses)
for job in mock_array_job.sub_jobs: print(job.qstat)

remaining jobs are [<MockPbsArrayJob name=mock_script id=[0]>, <PbsArraySubJob name=mock_script id=[0][2]>, <PbsArraySubJob name=mock_script id=[0][3]>]


AssertionError: 

In [ ]:
statuses=[{'status': 'R', 'job_id': '[0]', 'elap_time': '00:05'},{'status': 'R', 'job_id': '[0][3]', 'elap_time': '00:08'}]
print (statuses)

In [ ]:
mock_array_job.update_status(statuses)
for job in mock_array_job.sub_jobs: print(job.qstat)